# Backpropagation on DAGs

![Status](https://img.shields.io/static/v1.svg?label=Status&message=Finished&color=brightgreen)
[![Source](https://img.shields.io/static/v1.svg?label=GitHub&message=Source&color=181717&logo=GitHub)](https://github.com/particle1331/inefficient-networks/blob/master/docs/notebooks/fundamentals/backpropagation.ipynb)
[![Stars](https://img.shields.io/github/stars/particle1331/inefficient-networks?style=social)](https://github.com/particle1331/inefficient-networks)


---

## Introduction

In this notebook, we will look at **backpropagation** (BP) on **directed acyclic computational graphs** (DAG). Our main result is that a training step for a single data point (consisting of both a forward and a backward pass) has a time complexity that is linear in the number of edges of the network. 

<!-- In the last section, we take a closer look at the implementation of `.backward` in PyTorch. -->

**Readings**
* [Evaluating $\nabla f(x)$ is as fast as $f(x)$](https://timvieira.github.io/blog/post/2016/09/25/evaluating-fx-is-as-fast-as-fx/)
* [Back-propagation, an introduction](http://www.offconvex.org/2016/12/20/backprop/)
* [PyTorch Autograd Explained - In-depth Tutorial](https://www.youtube.com/watch?v=MswxJw-8PvE)

## Gradient descent on the loss surface

For every data point $(\mathbf x, y)$, the loss function $\ell$ assigns a nonnegative number $\ell(y, f_{\boldsymbol \Theta}(\mathbf x))$ that approaches zero whenever the predictions $f_{\boldsymbol \Theta}(\mathbf x)$ approach the target values $y$. Given the current parameters $\boldsymbol \Theta \in \mathbb R^d$ of a neural network $f,$ we can imagine the network to be at a certain point $(\boldsymbol \Theta, \mathcal L_{\mathcal X}(\boldsymbol \Theta))$ on a surface in $\mathbb R^d \times \mathbb R$ where $\mathcal L_{\mathcal X}(\boldsymbol \Theta)$ is the average loss over the dataset:

$$
\mathcal L_{\mathcal X}(\boldsymbol \Theta) = \frac{1}{|\mathcal X|} \sum_{(\mathbf x, y) \in \mathcal X} \ell(y, f_{\boldsymbol \Theta}(\mathbf x)).
$$

The empirical loss acts as an almost-everywhere differentiable surrogate to the true objective (e.g. a non-differentiable metric such as accuracy). This will generally vary for each sample, hence the subscript. Nevertheless, we except these surfaces to be very similar assuming the samples are drawn from the same distribution. Training a neural network is equivalent to finding the minimum of this surface. In practice, we use variants of gradient descent, characterized by the update rule $\boldsymbol \Theta \leftarrow \boldsymbol \Theta - \varepsilon \nabla_{\boldsymbol \Theta}\, \mathcal L_{\mathcal X}$ to find a local minimum where $-\nabla_{\boldsymbol \Theta}\, \mathcal L_{\mathcal X}$ is the direction of steepest descent at $\boldsymbol \Theta$ and  $\varepsilon > 0$ called the learning rate is a constant that controls the step size.


```{figure} ../../img/loss_surface_resnet.png
---
name: loss-surface-resnet
width: 35em
---
Loss surface for ResNet-56 with or without skip connections. Much of deep learning research is dedicated to studying the geometry of loss surfaces and its effect on optimization. {cite}`arxiv.1712.09913`
```



```{margin}
**The need for efficient BP**
```

Observe that $\nabla_{\boldsymbol \Theta}\, \mathcal L_{\mathcal X}$ consists of partial derivatives for each weight in the network. This can easily number in millions. To compute these values efficiently, we will perform computation in an efficient way. This improvement in time complexity is further supplemented with sophisticated hardware for parallel computation (e.g. GPUs) which can reduce training time by a significant factor resulting in the viability of neural network models for pratical uses.

## Backpropagation on Computational Graphs

A neural network can be modelled as a **directed acyclic graph** (DAG) of compute and parameter nodes that implements a function $f$ and can be extended to implement the calculation of the loss value for each training example and parameter values. In principle, we can perturb the current state of the network by perturbing the network weights. This results in perturbations flowing up to the final loss node assuming each computation is differentiable. In this section, we will look at an algorithm for computing derivatives of computational graphs.


A compute node simply implements a simple function of node values of nodes that are directed to it. Parameter nodes simply store values which ultimately determines the function $f.$ Note that since there are no cycles, and since the graph is directed, then we can write the network as a sequence of layers.

```{figure} ../../img/compute.svg
---
width: 30%
name: compute
---
Compute and parameter node as input to another compute node. Note that parameter nodes always have zero fan-in.
```

### Forward pass

To compute $f(\mathbf x)$, all compute nodes are executed up to the loss node following the directions and operations specified by the nodes and edges. 
The input $\mathbf x$ is passed to the input nodes and propagated forward through the network, computing and storing the output value of each node. The output values for compute nodes are stored to preserve the current state for backward pass, as well as to avoid recomputation for the nodes in the next layer. Assuming a node with $n$ inputs executes $n$ operations, then one forward pass takes $\mathcal{O}(E)$ calculations were $E$ is the number of edges of the graph. For neural networks the number of edges is proportional to the number of weights and activation units of the network.

### Backward pass

During backward pass gradients are categorized into two types: **local gradients** which are derivatives between adjacent nodes, and derivatives of the loss with respect to the node called **backpropagated gradients** since it is propagated from the top node backward into the current node. Recall from above that our goal is to calculate the backpropagated gradient of the loss with respect to each parameter node.

**BP proceeds recursively.** We will assume that the graph has been [topologically sorted](https://en.wikipedia.org/wiki/Topological_sorting). For neural networks, there is no need to do this since we already have a natural ordering defined by the layers. 
For the base step, the gradient of the compute node for the loss is set to $1$ (i.e. its derivative with itself) and stored in memory. Iterating backwards in the sorted list of nodes, we know that the backpropagated gradient for each compute node $v$ that depends on $u$ is already stored. Moreover, all local gradients between $u$ and $v$ are [computable at runtime](https://en.wikipedia.org/wiki/Automatic_differentiation). Then, the backpropagated gradient with respect to node $u$ can be calculated using the chain rule:

$$
{\frac{\partial\mathcal L}{\partial u} } = \sum_{ {v} } \left( {{\frac{\partial\mathcal L}{\partial v}}} \right) \left( {{\frac{\partial{v}}{\partial u}}} \right).
$$

Note that gradient type is distinguished by color in the figure below:


```{figure} ../../img/backward-1.svg
---
width: 80%
name: backward-1
---
Computing the backpropagated gradient for a single node.
```

This can be visualized as gradients "flowing" to each network node from the loss node. Observe that the flow of gradients end on parameter nodes since these nodes have zero fan-in. Here the partial derivatives are evaluated on the current network state with values obtained during forward pass. Hence, forward pass should always precede backward pass. Moreover, as required by the algorithm all backpropagated gradients are stored in each compute node for use by the next layer. Memory can be released after the weights are updated. On the other hand, there is no need to store local gradients; these are computed as needed. 
    
**Remark.** BP is a useful tool for understanding how derivatives flow through a model. This can be extremely helpful in reasoning about why some models are difficult to optimize. Classic examples are vanishing or exploding gradients as we go into deeper layers of the network.

### Training neural networks

Now that we know how to compute each backpropagated gradient, we can implement this as a method `u.backward()` for each node `u` that sends the gradient of the loss with respect to `u` to all nodes that depend on it. Every backpropagated gradient that is sent to a node is accumulated in a sum. Hence, we have to zero it out at the start of every trainng step. The [SGD algorithm](https://en.wikipedia.org/wiki/Stochastic_gradient_descent) with $\epsilon = 0.01$ for training a neural network is implemented below. Note that the input nodes selects a randomly takes a fixed-size sample of the dataset.

```python 
def Forward():
    input = sample_batch()
    for c in compute + input: 
        c.forward()

def Backward(loss):
    for c in compute + params: 
        c.grad = 0
    
    loss.grad = 1
    for c in compute[::-1]: 
        c.backward()

def SGD(eta):
    for w in params:
        w.value -= eta * w.grad


loss = epsilon + 1.0
while loss >= epsilon:
    Forward()
    Backward()
    SGD(eta=0.01)
```

<br>

Properties of the algorithm which makes it the practical choice for training huge neural networks are:

* **Modularity.** For neural networks, the dependence only on nodes belonging to the upper layer suggests a modularity in implementing neural networks, i.e. we can connect DAG subnetworks with possibly distinct network architectures by only connecting outermost nodes that are exposed between layers.

<br>

* **Efficiency.** Iterating over all nodes in the network during backward pass covers all the edges in the network with no edge counted twice. Assuming computing local gradients take constant time, then backward pass requires ${\mathcal O}(E)$ computations. For neural networks this is proportional to the number of neurons and parameters (i.e. the network size). Furthermore, the chain rule between nodes on the same layer and nodes in the upper layer of a network can be implemented as matrix multiplication for which there exist [highly optimized implementations](https://en.wikipedia.org/wiki/Basic_Linear_Algebra_Subprograms). This is done in the next section.

$\phantom{3}$

### BP equations for dense neural nets

```{margin}
Source:<br>**Figure 1** of {cite}`0483bd9444a348c8b59d54a190839ec9`
```
```{figure} ../../img/deep-nns.png
---
width: 40em
---

**Figure 1** of {cite}`0483bd9444a348c8b59d54a190839ec9` should hopefully make sense after reading this article. This figure shows (left) forward pass for a multilayer neural network, and (right) backward pass for the same network.
```

In this section, we solve the BP equations for a densely connected neural network. This can be useful for analyzing gradient flow. As shown in the above figure, a multilayer neural network can be modelled as a computational DAG with edges from preactivation to activation values and edges from weights and input values that fan into preactivations. Computation performed by the network at layer $t$ can be written in two steps as:

* ${\mathbf{y}}^{(t)} = {\mathbf{x}}^{(t-1)}{\boldsymbol{W}}^{(t)} + {\boldsymbol b}^{(t)}$ 
* ${\mathbf x}^{(t)} = \varphi({\mathbf y}^{(t)})$ 

Here we use row vectors for layer inputs and outputs. The following equations are obtained by simply matching input and output shapes, then trying to figure out the entries of the right hand side matrix by tracking node dependencies. For the compute nodes in the current layer, the BP equations are:
    
$$
\dfrac{\partial \mathcal L}{\partial {\mathbf x}^{(t)}} 
= 
\dfrac{\partial \mathcal L}{\partial {\mathbf y}^{(t+1)}} 
\dfrac{\partial {\mathbf y}^{(t+1)}}{\partial {\mathbf x}^{(t)}} 
=
\dfrac{\partial \mathcal L}{{\partial \mathbf{y}}^{(t+1)}}
\boldsymbol{W}^{(t+1)\top}
$$

and

$$
\dfrac{\partial \mathcal L}{\partial {{\mathbf y}}^{(t)}} 
= 
\dfrac{\partial \mathcal L}{\partial {\mathbf x}^{(t)}} 
\dfrac{\partial {\mathbf x}^{(t)}}{\partial {\mathbf y}^{(t)}} 
= 
\dfrac{\partial \mathcal L}{{\partial \mathbf{x}}^{(t)}}
{{\boldsymbol J}_\varphi}{^{(t)}}.
$$

Here the Jacobian matrix is defined as $\left({\boldsymbol J}_{\varphi}\right)_{ij} = \frac{\partial x_i}{\partial y_j}$. For commonly used activations, i.e. those which are applied entrywise on vector inputs, this matrix reduces to a diagonal matrix. For the parameter nodes, the backpropagated gradients are:

$$
\dfrac{\partial \mathcal L}{\partial {\boldsymbol W}^{(t)}} 
= 
\dfrac{\partial \mathcal L}{\partial {{\mathbf y}}^{(t)}} 
\dfrac{\partial {{\mathbf y}}^{(t)}}{\partial {\boldsymbol W}^{(t)}} 
= 
\mathbf{x}^{(t-1)\top}
\dfrac{\partial \mathcal L}{\partial {{\mathbf y}}^{(t)}} 
$$

and

$$
\dfrac{\partial \mathcal L}{\partial {\boldsymbol{b}_{j}}^{(t)}} 
= 
\dfrac{\partial \mathcal L}{\partial {{\mathbf y}}^{(t)}}
\dfrac{\partial {{\mathbf y}}^{(t)}}{\partial {\boldsymbol b}^{(t)}} 
= 
\dfrac{\partial \mathcal L}{\partial {{\mathbf y}}^{(t)}}.
$$

Backpropagated gradients for compute nodes have to be stored until weights are updated. Observe that derivatives of the compute nodes of layer $t+1$ are retrieved to compute gradients in layer $t.$ On the other hand, the local gradients ${\boldsymbol J}_\varphi$ are computed using autodifferentiation and are evaluated at the current network state based on values obtained at forward pass.

<br>

```{figure} ../../img/jacobian.svg
---
name: jacobian
width: 75%
---
Deriving the equations by matching shapes of derivatives as matrices. In general, we put the incoming gradients as input, and current gradients as output. What is left is to figure out the matrix in between containing the appropriate local derivatives.
```


### Computing derivatives with TensorFlow

The above equations and process is demonstrated in the following computation:

In [1]:
import tensorflow as tf
print(tf.__version__)

# Input and weight init.
x0 = tf.Variable(tf.random.normal(shape=(1, 4)))
W1 = tf.Variable(tf.random.normal(shape=(4, 3)))
b1 = tf.Variable(tf.random.normal(shape=(1, 3)))
W2 = tf.Variable(tf.random.normal(shape=(3, 2)))

2.8.0
Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



In [2]:
# Forward pass:
with tf.GradientTape(persistent=True) as tape:
    y1 = tf.matmul(x0, W1) + b1
    x1 = tf.keras.activations.softmax(y1)

    y2 = tf.matmul(x1, W2)
    loss = tf.reduce_sum(y2**2)

# Backward pass:
# (1) ∂L/∂x[t] = ∂L/∂y[t+1] W[t+1]. Fetch ∂L/∂y[t+1] from upper layer.
loss_x1 = tf.matmul(tape.gradient(loss, y2), tf.transpose(W2))

# (2) ∂L/∂y[t] = ∂L/∂x[t] J[t]. Compute local gradients using autodiff.
loss_y1 = tf.matmul(loss_x1, tf.reshape(tape.jacobian(x1, y1), (3, 3)))

# (3) ∂L/∂W[t] = x[t-1]T ∂L/∂y[t]
loss_W1 = tf.matmul(tf.transpose(x0), loss_y1)

# (4) ∂L/∂b[t] = ∂L/∂y[t]
loss_b1 = loss_y1

print("(weights)")
print("TF autodiff:")
print(tape.gradient(loss, W1).numpy())
print("\nBP equations:")
print(loss_W1.numpy())

print("\n(biases)")
print("TF autodiff: ", tape.gradient(L, b1).numpy())
print("BP equations:", loss_b1.numpy())

(weights)
TF autodiff:
[[-0.052407    0.10895263 -0.05654562]
 [-0.04306866  0.0895385  -0.04646983]
 [ 0.01126021 -0.02340965  0.01214944]
 [-0.03177317  0.0660555  -0.03428232]]

BP equations:
[[-0.052407    0.10895262 -0.05654562]
 [-0.04306867  0.08953848 -0.04646983]
 [ 0.01126021 -0.02340965  0.01214944]
 [-0.03177318  0.06605549 -0.03428232]]

(biases)
TF autodiff:  [[-0.03510599  0.07298434 -0.03787834]]
BP equations: [[-0.035106    0.07298433 -0.03787834]]


<!-- ## Autodifferentiation with PyTorch `autograd`

The `autograd` package allows automatic differentiation by building computational graphs on the fly every time we pass data through our model. Autograd tracks which data combined through which operations to produce the output. This allows us to take derivatives over ordinary imperative code. This functionality is consistent with the memory and time requirements outlined in above for BP.

<br>

**Backward for scalars.** Let $y = \mathbf x^\top \mathbf x = \sum_i {x_i}^2.$ In this example, we initialize a tensor `x` which initially has no gradient. Calling backward on `y` results in gradients being stored on the leaf tensor `x`.  -->

<!-- x = torch.arange(4, dtype=torch.float, requires_grad=True)
y = x.T @ x 

y.backward() 
(x.grad == 2*x).all() -->

<!-- **Backward for vectors.** Let $\mathbf y = g(\mathbf x)$ and let $\mathbf v$ be a vector having the same length as $\mathbf y.$ Then `y.backward(v)` implements   

$$\sum_i v_i \left(\frac{\partial y_i}{\partial x_j}\right)$$ 
  
resulting in a vector of same length as `x` that is stored in `x.grad`. Note that the terms on the right are the local gradients in backprop. Hence, if `v` contains backpropagated gradients of nodes that depend on `y`, then this operation gives us the backpropagated gradients with respect to `x`, i.e. setting $v_i = \frac{\partial \mathcal{L} }{\partial y_i}$ gives us the vector $\frac{\partial \mathcal{L} }{\partial x_j}.$ -->

<!-- x = torch.rand(size=(4,), dtype=torch.float, requires_grad=True)
v = torch.rand(size=(2,), dtype=torch.float)
y = x[:2]

# Computing the Jacobian by hand
J = torch.tensor(
    [[1, 0, 0, 0],
    [0, 1, 0, 0]], dtype=torch.float
)

# Confirming the above formula
y.backward(v)
(x.grad == v @ J).all() -->

<!-- **Locally disabling gradient tracking.** Disabling gradient computation is useful when computing values, e.g. accuracy, whose gradients will not be backpropagated into the network. To stop PyTorch from building computational graphs, we can put the code inside a `torch.no_grad()` context or inside a function with a `@torch.no_grad()` decorator.

Another technique is to use the `.detach()` method which returns a new tensor detached from the current graph but shares the same storage with the original one. In-place modifications on either of them will be seen, and may trigger errors in correctness checks. -->